In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI 模型花园 - OpenCLIP

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_pytorch_open_clip.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在 Colab 中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_pytorch_open_clip.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在 GitHub 上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/notebooks/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/model_garden/model_garden_pytorch_open_clip.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
在 Vertex AI Workbench 中打开
    </a>
  </td>
</table>

## 概述

本笔记本演示了使用[CC3M](https://ai.google.com/research/ConceptualCaptions/download)数据集对[OpenCLIP](https://github.com/mlfoundations/open_clip)进行微调，并在Vertex AI上部署在线预测。

### 目标

- 使用[Vertex AI自定义训练](https://cloud.google.com/vertex-ai/docs/training/overview)微调OpenCLIP模型。
- 将模型上传到[Vertex AI模型注册表](https://cloud.google.com/vertex-ai/docs/model-registry/introduction)。
- 将模型部署到[Vertex AI Endpoint资源](https://cloud.google.com/vertex-ai/docs/predictions/using-private-endpoints)。
- 运行零样本图像分类的在线预测。

### 成本

本教程使用Google Cloud的计费组件：

* Vertex AI
* Cloud Storage

了解[Vertex AI定价](https://cloud.google.com/vertex-ai/pricing)和[Cloud Storage定价](https://cloud.google.com/storage/pricing)，并使用[定价计算器](https://cloud.google.com/products/calculator/)根据您的预期使用情况生成费用估算。

## 设置环境

**注意**: Jupyter会将以`!`为前缀的行视为shell命令，并将以`$`为前缀的Python变量插入这些命令中。

### 仅限协作

In [ ]:
if "google.colab" in str(get_ipython()):
    ! pip3 install --upgrade google-cloud-aiplatform
    from google.colab import auth as google_auth

    google_auth.authenticate_user()

    # Restart the notebook kernel after installs.
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

### 设置 Google Cloud 项目

1. [选择或创建一个 Google Cloud 项目](https://console.cloud.google.com/cloud-resource-manager)。当您第一次创建帐户时，您将获得 $300 的免费信用额度用于您的计算/存储成本。

2. [确保您的项目已启用计费](https://cloud.google.com/billing/docs/how-to/modify-project)。

3. [启用 Vertex AI API 和 Compute Engine API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component)。

4. [创建一个 Cloud 存储存储桶](https://cloud.google.com/storage/docs/creating-buckets) 以存储实验输出。

5. [创建一个服务账号](https://cloud.google.com/iam/docs/service-accounts-create#iam-service-accounts-create-console) ，拥有 `Vertex AI 用户`、`存储对象管理员` 和 `GCS 存储桶所有者` 角色，用于部署优化模型到 Vertex AI 终端。

填写以下实验环境的变量:

In [ ]:
# Cloud project id.
PROJECT_ID = ""  # @param {type:"string"}

# The region you want to launch jobs in.
REGION = ""  # @param {type:"string"}

# The GCS bucket for storing experiments output. Fill it without the 'gs://' prefix.
GCS_BUCKET = ""  # @param {type:"string"}

# The service account for deploying fine tuned model.
# The service account looks like:
# '<account_name>@<project>.iam.gserviceaccount.com'
# Follow step 5 above to create this account.
SERVICE_ACCOUNT = ""  # @param {type:"string"}

### 下载数据到Google Cloud Storage

In [ ]:
# Install the library for downloading training data.
!pip install img2dataset==1.41.0

In [ ]:
import os

!gcloud storage cp gs://gcc-data/Validation/GCC-1.1.0-Validation.tsv ./data.tsv  # Download list of URLs.
!sed -i '1s/^/caption\turl\n/' data.tsv  # Add column name.

In [ ]:
# Download images from URLs. It takes around 2min.
output_folder = "data"
!img2dataset --url_list data.tsv --input_format "tsv"\
    --output_folder {output_folder}\
    --url_col "url" --caption_col "caption" --output_format webdataset\
    --processes_count {os.cpu_count()} --thread_count {os.cpu_count()*4}\
    --image_size 256

In [ ]:
GCS_data_dir = f"{GCS_BUCKET}/CC3M-val-wds"

In [ ]:
# Upload data to GCS.
!gcloud storage cp -r data gs://{GCS_data_dir}

In [ ]:
# Get total number of samples, which is required by OpenCLIP training.
import json

n_samples = 0
for filename in [f for f in os.listdir(output_folder) if f.endswith("_stats.json")]:
    with open(os.path.join(output_folder, filename)) as f:
        n_samples += json.load(f)["successes"]
print(n_samples)

初始化用于 Python 的 Vertex AI SDK

为您的项目和相应的存储桶初始化用于 Python 的 Vertex AI SDK。

In [ ]:
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=GCS_BUCKET)

定义常量

In [ ]:
# The pre-built training docker image. It contains training scripts and models.
TRAIN_DOCKER_URI = "us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/pytorch-open-clip-train"

# The pre-built serving docker image. It contains serving scripts and models.
SERVE_DOCKER_URI = "us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/pytorch-open-clip-serve"

### 为任务操作和测试数据准备定义常用函数

In [ ]:
import base64
from datetime import datetime
from io import BytesIO

import matplotlib.pyplot as plt
import requests
from PIL import Image


def create_job_name(prefix):
    """Create a job name string with a prefix."""
    user = os.environ.get("USER")
    now = datetime.now().strftime("%Y%m%d_%H%M%S")
    job_name = f"{prefix}-{user}-{now}"
    return job_name


def deploy_model(checkpoint, model, precision, task):
    """Deploy a model to Vertex AI endpoint."""
    model_name = "openclip"
    endpoint = aiplatform.Endpoint.create(display_name=f"{model_name}-{task}-endpoint")
    serving_env = {
        "CHECKPOINT": checkpoint,
        "MODEL_ID": model,
        "PRECISION": precision,
        "TASK": task,
        "DEPLOY_SOURCE": "notebook",
    }
    model = aiplatform.Model.upload(
        display_name=model_name,
        serving_container_image_uri=SERVE_DOCKER_URI,
        serving_container_ports=[7080],
        serving_container_predict_route="/predictions/transformers_serving",
        serving_container_health_route="/ping",
        serving_container_environment_variables=serving_env,
    )
    model.deploy(
        endpoint=endpoint,
        machine_type="n1-standard-4",
        accelerator_type="NVIDIA_TESLA_V100",
        accelerator_count=1,
        deploy_request_timeout=1800,
        service_account=SERVICE_ACCOUNT,
    )
    return model, endpoint


def download_image(url):
    response = requests.get(url)
    return Image.open(BytesIO(response.content)).convert("RGB")


def image_to_base64(image):
    buffer = BytesIO()
    image.save(buffer, format="PNG")
    image_str = base64.b64encode(buffer.getvalue()).decode("utf-8")
    return image_str


def plot_images(images, rows=1, cols=None):
    fig, axes = plt.subplots(rows, cols if cols is not None else len(images))
    for ax, img in zip(axes, images):
        ax.imshow(img)
        ax.axis("off")

**Fine tune the model**

本节对OpenCLIP模型进行微调，使用[CC3M](https://ai.google.com/research/ConceptualCaptions/download)数据集验证集来进行图像文本预训练。它加载OpenAI预训练的检查点。在这里，您使用一个小型的***RN50***模型；请查看[此列表](https://github.com/mlfoundations/open_clip#pretrained-model-interface)以获取OpenAI检查点支持的其他选项，例如***RN50, ViT-B-32等***。

需要一台带有1个`NVIDIA_TESLA_V100`的`n1-standard-4`机器来运行微调作业。微调作业大约需要3分钟完成2个时期的训练。

微调后的模型将在作业完成后保存，然后可以加载用于推断。

In [ ]:
machine_type = "n1-standard-4"
gpu_type = "NVIDIA_TESLA_V100"
num_gpus = 1

job_name = create_job_name("openclip")

model_name = "RN50"
precision = "amp"

job = aiplatform.CustomContainerTrainingJob(
    display_name=job_name, container_uri=TRAIN_DOCKER_URI, command=["torchrun"]
)

job.run(
    args=[
        f"--nproc_per_node={num_gpus}",
        "-m",
        "training.main",
        f"--name={job_name}",
        f"--logs=/gcs/{GCS_BUCKET}",
        f"--train-data=/gcs/{GCS_data_dir}/{{00000..00001}}.tar",
        f"--train-num-samples={n_samples}",
        "--dataset-type=webdataset",
        "--batch-size=32",
        "--precision=amp",
        "--workers=8",
        "--dataset-resampled",
        "--save-frequency=2",
        "--epochs=2",
        f"--model={model_name}",
        "--pretrained=openai",
        "--save-most-recent",
    ],
    boot_disk_size_gb=600,
    replica_count=1,
    machine_type=machine_type,
    accelerator_type=gpu_type,
    accelerator_count=num_gpus,
)

上传和部署模型

该部分将精调的模型上传到模型注册表并部署在端点上。

需要一个带有1个`NVIDIA_TESLA_V100`的`n1-standard-4`机器来部署OpenCLIP模型。

模型部署步骤大约需要20分钟完成。

In [ ]:
# Prepare image samples
img_diagram = download_image(
    "https://raw.githubusercontent.com/mlfoundations/open_clip/main/docs/CLIP.png"
)
img_cat = download_image(
    "https://upload.wikimedia.org/wikipedia/commons/thumb/9/9d/Calicocats2222.jpg/220px-Calicocats2222.jpg"
)
plot_images([img_diagram, img_cat])

#### 零样本图像分类

In [ ]:
model, endpoint = deploy_model(
    checkpoint=f"gs://{GCS_BUCKET}/{job_name}/checkpoints/epoch_latest.pt",
    model=model_name,
    precision=precision,
    task="zero-shot-image-classification",
)

请注意：模型权重在部署成功后下载。在上述模型部署步骤成功之后，需要额外等待5分钟，然后再运行下面的下一个步骤。否则，在向端点发送请求时可能会出现“ServiceUnavailable: 503 502:Bad Gateway”错误。

In [ ]:
instances = [
    {
        "text": ["a diagram", "a dog", "a cat"],
        "image": image_to_base64(img_diagram),
    },
    {
        "text": ["a diagram", "a dog", "two cats", "calico cat"],
        "image": image_to_base64(img_cat),
    },
]
response = endpoint.predict(instances=instances).predictions
response

清理资源:

In [ ]:
# Undeploy model and delete endpoint.
endpoint.delete(force=True)

# Delete models.
model.delete()

图像/文本特征嵌入

In [ ]:
model, endpoint = deploy_model(
    checkpoint=f"gs://{GCS_BUCKET}/{job_name}/checkpoints/epoch_latest.pt",
    model=model_name,
    precision=precision,
    task="feature-embedding",
)

注意：模型权重在部署成功后下载。在上述模型部署步骤成功后，需要额外等待5分钟**之后**才能运行下面的下一步。否则，当您向终端点发送请求时，可能会看到`ServiceUnavailable: 503 502:Bad Gateway`错误。

In [ ]:
import numpy as np

instances = [
    {
        "text": ["a diagram", "a dog", "a cat"],
        "image": image_to_base64(img_diagram),
    },
    {
        "image": image_to_base64(img_cat),
    },
    {
        "text": ["a diagram", "a dog", "two cats", "calico cat"],
    },
    {
        "text": "a single value",
    },
]
response = endpoint.predict(instances=instances).predictions
for pred in response:
    for k, v in pred.items():
        print(k, np.array(v).shape)

清理资源。

In [ ]:
# Undeploy model and delete endpoint.
endpoint.delete(force=True)

# Delete models.
model.delete()